# Entanglement by measurement on Starmon-5

In [1]:
from typing import List
from tqdm import tqdm
import numpy as np
import itertools as it

import qiskit
import qiskit.tools.jupyter
from qiskit import QuantumCircuit
import qiskit.circuit.library.standard_gates as gates
import qiskit.tools.visualization as qvis

from importlib import reload
import inspire_experiments
reload(inspire_experiments)
from inspire_experiments import ExperimentData, get_starmon_status, inspire_login, get_file_header, measure_readout_correction

# %matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2
%qiskit_version_table

In [2]:
api, backend = inspire_login()
backend.status()

Backend status:  IDLE


In [3]:
get_starmon_status(api)

'IDLE'

## Measure


In [4]:
def measure_entanglement_by_msmt(
        qubits: list[int],
        tomo_qubits: list[int] = [],
        sim_msmt: bool = True,
        dd_sequence: str = None,
        exp_basename: str = "entanglement_by_msmt",
        extra_label: str = None,
        show_circuit: bool = False,
        send_jobs: bool = True
):
    jobs = []
    circuits = []
    if tomo_qubits:
        tomo_gates = {'X': gates.RYGate(-np.pi/2), 'Y': gates.RXGate(np.pi/2), 'Z': gates.IGate()}
        tomo_combinations = list(it.product(tomo_gates.keys(), repeat=len(tomo_qubits)))
    else:
        tomo_combinations = ['Z']

    for tomo_option in tqdm(tomo_combinations, total=len(tomo_combinations)):
        circuit = QuantumCircuit(5,5)
        # circuit.rx(np.pi/2, qubits[1])
        # circuit.ry(-np.pi/2, qubits[0])
        # circuit.ry(-np.pi/2, qubits[2])

        circuit.x(qubits[0])

        for qb in qubits:
            circuit.ry(np.pi/2, qb)

        circuit.barrier(range(5))
        circuit.cz(qubits[1], qubits[0])
        circuit.id(qubits[1])
        # circuit.id(qubits[1])
        circuit.cz(qubits[1], qubits[2])
        circuit.barrier(range(5))

        for qb in qubits:
            circuit.ry(-np.pi/2, qb)

        # circuit.rx(np.pi/2, qubits[1])
        # circuit.ry(np.pi/2, qubits[0])
        # circuit.ry(np.pi/2, qubits[2])
        circuit.barrier(range(5))
        if qubits[1] in tomo_qubits:
            idx = tomo_qubits.index(qubits[1])
            circuit.append(tomo_gates[tomo_option[idx]], [qubits[1]])
        elif sim_msmt:
            circuit.id(qubits[1])
        circuit.measure(qubits[1], qubits[1])


        # for qb in set(qubits).intersection(tomo_qubits):
        #     idx = tomo_qubits.index(qb)
        #     circuit.append(tomo_gates[tomo_option[idx]], [qb])
        #     circuit.measure(qb, qb)

        if not sim_msmt:
            if dd_sequence:
                # add decoupling sequence XY4, padded with 2 IDs at beginning and end
                for _ in range(2):
                    circuit.id(qubits[0])
                    circuit.id(qubits[-1])
                for _ in range(1):
                    for _ in range(4):
                        circuit.x(qubits[0])
                        circuit.x(qubits[-1])
                        circuit.y(qubits[0])
                        circuit.y(qubits[-1])
                for _ in range(2):
                    circuit.id(qubits[0])
                    circuit.id(qubits[-1])
                circuit.barrier(range(5))
            else:
                # add waiting time via ID gates on data qubits
                for _ in range(100):
                    circuit.id(qubits[0])
                    circuit.id(qubits[-1])


        # for qb in set(qubits).difference(tomo_qubits):
        #     circuit.id(qb)
        #     circuit.measure(qb, qb)

        if qubits[0] in tomo_qubits:
            idx = tomo_qubits.index(qubits[0])
            circuit.append(tomo_gates[tomo_option[idx]], [qubits[0]])
        elif sim_msmt:
            circuit.id(qubits[0])
        circuit.measure(qubits[0], qubits[0])

        if qubits[2] in tomo_qubits:
            idx = tomo_qubits.index(qubits[2])
            circuit.append(tomo_gates[tomo_option[idx]], [qubits[2]])
        elif sim_msmt:
            circuit.id(qubits[2])
        circuit.measure(qubits[2], qubits[2])

        if show_circuit:
            display(circuit.draw(output='mpl', scale=1, fold=50))
            # qvis.circuit_drawer(circuit, output='mpl', scale=0.5, line_length=-1)
        circuits += [circuit]

        exp_name = exp_basename + f"_qbs{qubits}"
        exp_name += "_sim-msmt" if sim_msmt else ''
        exp_name += f"_{dd_sequence}" if dd_sequence else ''
        exp_name += extra_label if extra_label else ''
        exp_name += f"_tom{tomo_qubits}" + f"_M{tomo_option}" if tomo_qubits else ''
        print(exp_name)

        if send_jobs:
            header = get_file_header(circuit)
            print(get_starmon_status(api))
            job = qiskit.execute(circuit, shots=2**14, optimization_level=0, backend=backend)
            ExperimentData.save_job_result(job, exp_name, header)
            jobs += [job]

    return jobs, circuits


In [6]:
jobs, circuits = measure_entanglement_by_msmt([0,2,4], tomo_qubits=[0,4], extra_label="_Xqb0", dd_sequence='', sim_msmt=False, show_circuit=False, send_jobs=True)

  0%|          | 0/9 [00:00<?, ?it/s]

entanglement_by_msmt_qbs[0, 2, 4]_Xqb0_tom[0, 4]_M('X', 'X')
IDLE


01:07:51 - inspire_experiments.data_utils:INFO - Saving results for job 7166471 in ./data/
 11%|█         | 1/9 [00:50<06:47, 50.88s/it]

entanglement_by_msmt_qbs[0, 2, 4]_Xqb0_tom[0, 4]_M('X', 'Y')
IDLE


01:08:42 - inspire_experiments.data_utils:INFO - Saving results for job 7166472 in ./data/
 22%|██▏       | 2/9 [01:38<05:41, 48.81s/it]

...

01:12:40 - inspire_experiments.data_utils:INFO - Saving results for job 7166477 in ./data/
 78%|███████▊  | 7/9 [05:36<01:35, 47.60s/it]

entanglement_by_msmt_qbs[0, 2, 4]_Xqb0_tom[0, 4]_M('Z', 'Y')
IDLE


01:13:28 - inspire_experiments.data_utils:INFO - Saving results for job 7166478 in ./data/
 89%|████████▉ | 8/9 [06:24<00:47, 47.76s/it]

entanglement_by_msmt_qbs[0, 2, 4]_Xqb0_tom[0, 4]_M('Z', 'Z')
EXECUTION


01:14:16 - inspire_experiments.data_utils:INFO - Saving results for job 7166479 in ./data/
100%|██████████| 9/9 [07:12<00:00, 48.08s/it]


In [24]:
jobs2, circuits2 = measure_entanglement_by_msmt([0,2,4], tomo_qubits=[0,4], extra_label="_Xqb0", send_jobs=True, sim_msmt=True, show_circuit=False)

  0%|          | 0/9 [00:00<?, ?it/s]

entanglement_by_msmt_qbs[0, 2, 4]_sim-msmt_Xqb0_tom[0, 4]_M('X', 'X')
EXECUTION


18:34:55 - inspire_experiments.data_utils:INFO - Saving results for job 7166448 in ./data/
 11%|█         | 1/9 [00:46<06:13, 46.70s/it]

entanglement_by_msmt_qbs[0, 2, 4]_sim-msmt_Xqb0_tom[0, 4]_M('X', 'Y')
EXECUTION


18:35:41 - inspire_experiments.data_utils:INFO - Saving results for job 7166449 in ./data/
 22%|██▏       | 2/9 [01:35<05:34, 47.74s/it]

...

18:41:16 - inspire_experiments.data_utils:INFO - Saving results for job 7166454 in ./data/
 78%|███████▊  | 7/9 [07:55<02:42, 81.38s/it]

entanglement_by_msmt_qbs[0, 2, 4]_sim-msmt_Xqb0_tom[0, 4]_M('Z', 'Y')
EXECUTION


18:42:50 - inspire_experiments.data_utils:INFO - Saving results for job 7166455 in ./data/
 89%|████████▉ | 8/9 [09:31<01:26, 86.23s/it]

entanglement_by_msmt_qbs[0, 2, 4]_sim-msmt_Xqb0_tom[0, 4]_M('Z', 'Z')
IDLE


18:44:26 - inspire_experiments.data_utils:INFO - Saving results for job 7166456 in ./data/
100%|██████████| 9/9 [11:05<00:00, 73.95s/it]


## readout correction calibration points

In [25]:
measure_readout_correction(qubits=[0,4], extra_label="_Xqb0", backend=backend, send_jobs=True, show_circuit=False)

  0%|          | 0/4 [00:00<?, ?it/s]18:45:59 - inspire_experiments.experiment_library:INFO - readout_correction_qbs[0, 4]_Xqb0_state00
18:46:00 - inspire_experiments.experiment_library:INFO - Backend state: EXECUTION
18:46:00 - inspire_experiments.experiment_library:INFO - Measuring readout correction: state ('0', '0')
18:46:00 - inspire_experiments.data_utils:INFO - Saving results for job 7166457 in ./data/
 25%|██▌       | 1/4 [01:35<04:45, 95.29s/it]18:47:35 - inspire_experiments.experiment_library:INFO - readout_correction_qbs[0, 4]_Xqb0_state01
18:47:35 - inspire_experiments.experiment_library:INFO - Backend state: IDLE
18:47:35 - inspire_experiments.experiment_library:INFO - Measuring readout correction: state ('0', '1')
18:47:35 - inspire_experiments.data_utils:INFO - Saving results for job 7166458 in ./data/
 50%|█████     | 2/4 [03:09<03:09, 94.51s/it]18:49:09 - inspire_experiments.experiment_library:INFO - readout_correction_qbs[0, 4]_Xqb0_state10
18:49:09 - inspire_experime

([<quantuminspire.qiskit.qi_job.QIJob at 0x7f2df3f09b70>,
  <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7f2df3f09f60>])

In [12]:
dir(backend)


['DEFAULT_CONFIGURATION',
 '_QuantumInspireBackend__api',
 '_QuantumInspireBackend__backend',
 '_QuantumInspireBackend__convert_histogram',
 '_QuantumInspireBackend__convert_result_data',
 '_QuantumInspireBackend__convert_result_multiple_shots',
 '_QuantumInspireBackend__convert_result_single_shot',
 '_QuantumInspireBackend__raw_qubit_register_to_raw_data_value',
 '_QuantumInspireBackend__validate_full_state_projection',
 '_QuantumInspireBackend__validate_nr_of_clbits_conditional_gates',
 '_QuantumInspireBackend__validate_number_of_shots',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_configuration',
 '_default_op